<a href="https://colab.research.google.com/github/s-ryuri/TIL/blob/main/%EA%B3%B5%EB%AA%A8%EC%A0%84EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Colab/9월16일

import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import warnings
#ignore warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn')

plt.rc('font', family='NanumBarunGothic') 
def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[0]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
change_matplotlib_font(font_download_url)

In [ ]:
powerfactor = pd.read_json('347_역률평균.json')
powerfactor

# 데이터 불러옴

In [ ]:
electric = pd.read_json('347_전류고조파평균.json')
electric

# 데이터 불러옴

In [ ]:
voltage = pd.read_json('347_전압고조파평균.json')
voltage

# 데이터 불러옴

In [ ]:
powerlist = []
def powerapply(col):
    powerlist.append(list(col.values()))
    return list(col.values())

power_temp = powerfactor['data'].apply(powerapply)
column_name = ['ITEM_NAME','ITEM_VALUE','LABEL_NAME','TIMESTAMP']

power_label = pd.DataFrame.from_records(powerlist,columns = column_name)

powerfactor = pd.concat([powerfactor,power_label],axis = 1)
powerfactor.drop('data',axis = 1,inplace = True)

powerfactor.rename(columns = {'LABEL_NAME' : 'TIMESTAMP','TIMESTAMP' : 'LABEL_NAME'},inplace = True)
powerfactor

# data에 dict형태로 되어있는 걸 꺼내서 column으로 추가해줌

eleclist = []
def elecapply(col):
    eleclist.append(list(col.values()))
    return list(col.values())

elec_temp = electric['data'].apply(elecapply)
column_name = ['ITEM_NAME','ITEM_VALUE','LABEL_NAME','TIMESTAMP']

elec_label = pd.DataFrame.from_records(eleclist,columns = column_name)

electric = pd.concat([electric,elec_label],axis = 1)
electric.drop('data',axis = 1,inplace = True)

electric.rename(columns = {'LABEL_NAME' : 'TIMESTAMP','TIMESTAMP' : 'LABEL_NAME'},inplace = True)
electric

# data에 dict형태로 되어있는 걸 꺼내서 column으로 추가해줌

voltlist = []
def voltapply(col):
    voltlist.append(list(col.values()))
    return list(col.values())

volt_temp = voltage['data'].apply(voltapply)
column_name = ['ITEM_NAME','ITEM_VALUE','LABEL_NAME','TIMESTAMP']

volt_label = pd.DataFrame.from_records(voltlist,columns = column_name)

voltage = pd.concat([voltage,volt_label],axis = 1)
voltage.drop('data',axis = 1,inplace = True)

voltage.rename(columns = {'LABEL_NAME' : 'TIMESTAMP','TIMESTAMP' : 'LABEL_NAME'},inplace = True)
voltage

# data에 dict형태로 되어있는 걸 꺼내서 column으로 추가해줌

In [ ]:
powerfactor = powerfactor[['BASE_ITEM','ITEM_NAME', 'ITEM_VALUE', 'TIMESTAMP', 'LABEL_NAME']]
electric = electric[['BASE_ITEM','ITEM_NAME', 'ITEM_VALUE', 'TIMESTAMP', 'LABEL_NAME']]
voltage = voltage[['BASE_ITEM','ITEM_NAME', 'ITEM_VALUE', 'TIMESTAMP', 'LABEL_NAME']]

# 일단 BASE_TIME, ITEM_NAME, ITEM_VALUE, TIMESTAMP, LABEL_NAME만 꺼내줌
# LABEL_NAME이 타겟값임

In [ ]:
powerfactor

In [ ]:
def second(col):
    return col[:16]

powerfactor['TIMESTAMP'] = powerfactor['TIMESTAMP'].apply(second)
electric['TIMESTAMP'] = electric['TIMESTAMP'].apply(second)
voltage['TIMESTAMP'] = voltage['TIMESTAMP'].apply(second)

# 00:00:13에서 초를 없애줌

In [ ]:
powerfactor

In [ ]:
fig, ax = plt.subplots(3,1,figsize = (18,18))


powerfactor['LABEL_NAME'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0], rot = 0)
ax[0].set_title('역률 평균의 경고, 정상, 주의')
ax[0].set_ylabel('개수')
plt.xticks(rotation = 90)
electric['LABEL_NAME'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[1],rot = 0)
ax[1].set_title('전류고조파평균의 경고, 정상, 주의')
ax[1].set_ylabel('개수')
voltage['LABEL_NAME'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[2], rot = 0)
ax[2].set_title('전압고조파평균의 경고, 정상, 주의')
ax[2].set_ylabel('개수')

In [ ]:
electric['LABEL_NAME'].value_counts()

In [ ]:
idx = powerfactor[(powerfactor['LABEL_NAME'] != electric['LABEL_NAME']) & (powerfactor['LABEL_NAME'] != voltage['LABEL_NAME']) & (voltage['LABEL_NAME'] != electric['LABEL_NAME'])].index
len(idx)

# 역률, 전류, 전압 LABEL_NAME이 다 다른 것
# 1 : 1 : 1

In [ ]:
powerfactor.loc[14315]['LABEL_NAME']

In [ ]:
electric.loc[14315]['LABEL_NAME']

In [ ]:
voltage.loc[14315]['LABEL_NAME']

In [ ]:
powerfactor

In [ ]:
electric

In [ ]:
makecolumn = voltage['ITEM_NAME'].unique()

In [ ]:
data = pd.DataFrame(columns = makecolumn)

In [ ]:

data['TIMESTAMP'] = 0
data['LABEL_NAME'] = 0

In [ ]:
data

In [ ]:
time = electric['TIMESTAMP'].unique()

In [ ]:
time

In [ ]:
data['TIMESTAMP'] = time

In [ ]:
data

In [ ]:
datacol = voltage[voltage['TIMESTAMP'] == '2021-01-16 00:00'].sort_values(by = 'ITEM_NAME')['ITEM_NAME']
datacol = list(datacol)

datacol.append('TIMESTAMP')
datacol.append('LABEL_NAME')

data.columns = datacol
data

In [ ]:
timecols = voltage['TIMESTAMP'].unique()
timecols


for time, col in enumerate(timecols):
    temp = voltage[voltage['TIMESTAMP'] == col].sort_values(by = 'ITEM_NAME')
    temp = temp.drop_duplicates()
    temp2 = temp['ITEM_VALUE']
    data.iloc[time,:35] = list(temp2)
    data.loc[time,'LABEL_NAME'] = temp['LABEL_NAME'].unique()[0]

In [ ]:
data.to_csv('new전압평균',index = False)

In [ ]:
a = pd.read_csv('new전압평균')
a

In [ ]:
a['LABEL_NAME'].value_counts()